In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from elastic import elastic_session, ScrollingCorpus, Document
from sentence_transformers import SentenceTransformer
from sentence import doc_to_sentences

2025-03-30 02:31:18.380119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743294678.393547   52088 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743294678.397367   52088 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
session = elastic_session("arxiv-index", "../credentials.json", "../http_ca.crt")
doc = Document(session, 0, text_path="article")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentences = doc_to_sentences(doc, model)

In [3]:
from itertools import starmap, pairwise
from rich.table import Table
from rich.console import Console
from rich.markdown import Markdown

def print_pairs(sentences):
    console = Console()
    console.clear()

    table = Table()
    table.add_column("Sentence Pair")
    table.add_column("Similarity", vertical="top")

    for thing in starmap(lambda x, y: (x,y,x.sim(y)), pairwise(sentences)):
        mytext = f'''
- {thing[0].text.strip()}


- {thing[1].text.strip()}

---
        '''
        table.add_row(Markdown(mytext), "\n\n"+str(thing[2]))

    console.print(table)

sentences = sentences[:20]
print_pairs(sentences)







┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sentence Pair                                                                                      ┃ Similarity ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│                                                                                                    │            │
│  • for about 20 years the problem of properties of short - term changes of solar activity has been │            │
│    considered extensively .                                                                        │ 0.77218753 │
│  • many investigators studied the short - term periodicities of the various indices of solar       │            │
│    activity .                                                                                      │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • many investigators studied the short - term periodicities of the various indices of solar       │            │
│    activity .                                                                                      │ 0.44803652 │
│  • several periodicities were detected , but the periodicities about 155 days and from the         │            │
│    interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .                   │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • several periodicities were detected , but the periodicities about 155 days and from the         │            │
│    interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .                   │ 0.40773794 │
│  • first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by  │            │
│    the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .                       │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by  │            │
│    the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .                       │ 0.60403246 │
│  • this periodicity was confirmed for other solar flares data and for the same time period @xcite  │            │
│    .                                                                                               │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • this periodicity was confirmed for other solar flares data and for the same time period @xcite  │            │
│    .                                                                                               │ 0.72166276 │
│  • it was also found in proton flares during solar cyc

In [4]:
from sentence import SimilarityPair, SentenceChain, Sentence
from itertools import chain

def iterative_merge(sentences, threshold):

    pairs = [SimilarityPair.from_sentences(s1, s2) for s1, s2 in pairwise(sentences)]

    '''
    def merge(pair: SimilarityPair):
        if pair.sim >= threshold:
            yield SentenceChain(pair)
        else:
            yield pair.s1
            yield pair.s2

    return chain.from_iterable(map(merge, pairs))'
    '''

    chains = [[]]

    for i, pair in enumerate(pairs):
        if pair.sim >= threshold: #Add to the chain
            if i == 0:
                chains[-1] += [pair.s1, pair.s2]
            else:
                #We have already examined s1
                chains[-1] += [pair.s2]

        else: #Create new chain for this sentence
            if i == 0:
                chains.append([pair.s1, pair.s2])
            else:
                #We have already examined s1
                chains.append([pair.s2])

    return [SentenceChain(c) for c in chains]

merged = iterative_merge(sentences, 0.5)

for thing in merged:
    print(thing)

["for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively .", "many investigators studied the short - term periodicities of the various indices of solar activity ."]
["several periodicities were detected , but the periodicities about 155 days and from the interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often ."]
["first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .", "this periodicity was confirmed for other solar flares data and for the same time period @xcite .", "it was also found in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar flares data during solar cycles 22 @xcite .", "_    several autors confirmed above results for the daily sunspot area data . @xcite studied the sunspot data from 18741984 ."]
["she found the 155-day periodicity in data 

In [5]:
print_pairs(merged)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sentence Pair                                                                                      ┃ Similarity ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│                                                                                                    │            │
│  • for about 20 years the problem of properties of short - term changes of solar activity has been │            │
│    considered extensively . many investigators studied the short - term periodicities of the       │ 0.4055183  │
│    various indices of solar activity .                                                             │            │
│  • several periodicities were detected , but the periodicities about 155 days and from the         │            │
│    interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .                   │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • several periodicities were detected , but the periodicities about 155 days and from the         │            │
│    interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .                   │ 0.5364079  │
│  • first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by  │            │
│    the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) . this periodicity was  │            │
│    confirmed for other solar flares data and for the same time period @xcite . it was also found   │            │
│    in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar       │            │
│    flares data during solar cycles 22 @xcite . _    several autors confirmed above results for the │            │
│    daily sunspot area data . @xcite studied the sunspot data from 18741984 .                       │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by  │            │
│    the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) . this periodicity was  │ 0.43505338 │
│    confirmed for other solar flares data and for the same time period @xcite . it was also found   │            │
│    in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar       │            │
│    flares data during solar cycles 22 @xcite . _    several autors confirmed above results for the │            │
│    daily sunspot area data . @xcite studied the sunspot data from 18741984 .                       │            │
│  • she found the 155-day periodicity in data records from 31 years .                               │            │
│                                                                                                    │            │
│ ────────────────────────────────────────────────────────────────────────────────────────────────── │            │
│                                                                                                    │            │
│  • she found the 155-day periodicity in data records from 31 years .                               │            │
│  • this periodicity is always characteristic for one o